<a href="https://colab.research.google.com/github/Dalvisneha12/IBM_AIMI_Cloud_python/blob/main/Sneha_diabetic_retinopathy_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q kaggle

In [ ]:
!mkdir ~/.kaggle # creating a kaggle directory

In [11]:
pwd

'/content/kaggle'

In [12]:
ls

kaggle.json


In [13]:
!unzip /content/kaggle.zip


Archive:  /content/kaggle.zip
  inflating: kaggle/kaggle.json      


In [14]:
cd /content/kaggle

/content/kaggle


In [6]:
ls

kaggle.json


In [15]:
!cp kaggle.json ~/.kaggle/  # copying json file to folder

In [16]:
!chmod 600 ~/.kaggle/kaggle.json  # changing the permissions to json

In [17]:
!kaggle datasets download -d arbethi/diabetic-retinopathy-level-detection

100% 9.65G/9.66G [01:27<00:00, 110MB/s]
100% 9.66G/9.66G [01:27<00:00, 118MB/s]


In [27]:
!unzip diabetic-retinopathy-level-detection.zip

Archive:  diabetic-retinopathy-level-detection.zip
  inflating: inception-diabetic.h5   
  inflating: preprocessed dataset/preprocessed dataset/testing/0/cfb17a7cc8d4.png  
  inflating: preprocessed dataset/preprocessed dataset/testing/0/cfdbaef73a8b.png  
  inflating: preprocessed dataset/preprocessed dataset/testing/0/cfed7c1172ec.png  
  inflating: preprocessed dataset/preprocessed dataset/testing/0/cff262ed8f4c.png  
  inflating: preprocessed dataset/preprocessed dataset/testing/0/cffc50047828.png  
  inflating: preprocessed dataset/preprocessed dataset/testing/0/d02b79fc3200.png  
  inflating: preprocessed dataset/preprocessed dataset/testing/0/d0926ed2c8e5.png  
  inflating: preprocessed dataset/preprocessed dataset/testing/0/d160ebef4117.png  
  inflating: preprocessed dataset/preprocessed dataset/testing/0/d16e39b9d6f0.png  
  inflating: preprocessed dataset/preprocessed dataset/testing/0/d16e59a2b33a.png  
  inflating: preprocessed dataset/preprocessed dataset/testing/0/d18f64

In [28]:
from tensorflow.keras.layers import Dense, Flatten, Input
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.applications.xception import Xception, preprocess_input
from glob import glob
import numpy as np
import matplotlib.pyplot as plt

In [29]:
pwd

'/content/kaggle'

In [30]:
ls

 diabetic-retinopathy-level-detection.zip   kaggle/      'preprocessed dataset'/
 inception-diabetic.h5                      kaggle.json


In [32]:
cd /content/kaggle/preprocessed dataset/preprocessed dataset

/content/kaggle/preprocessed dataset/preprocessed dataset


In [33]:
imageSize = [299, 299]

trainPath = r"/content/kaggle/preprocessed dataset/preprocessed dataset/training"

testPath = r"/content/kaggle/preprocessed dataset/preprocessed dataset/testing"

In [34]:
xception = Xception(input_shape=imageSize + [3], weights='imagenet',include_top=False)

83683744/83683744 [==============================] - 1s 0us/step


In [35]:
# don't train existing weights
for layer in xception.layers:
    layer.trainable = False

In [36]:
# our layers - you can add more if you want
x = Flatten()(xception.output)

In [37]:
prediction = Dense(5, activation='softmax')(x)

In [38]:
# create a model object
model = Model(inputs=xception.input, outputs=prediction)

In [39]:
# tell the model what cost and optimization method to use
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)


In [40]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [42]:
pwd

'/content/kaggle/preprocessed dataset/preprocessed dataset'

In [43]:
ls

testing/  training/


In [44]:
training_set = train_datagen.flow_from_directory(r'/content/kaggle/preprocessed dataset/preprocessed dataset/training',
                                                 target_size = (299, 299),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

test_set = test_datagen.flow_from_directory(r'/content/kaggle/preprocessed dataset/preprocessed dataset/testing',
                                            target_size = (299, 299),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 3662 images belonging to 5 classes.
Found 734 images belonging to 5 classes.


In [45]:
# fit the model
r = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=5,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)

<ipython-input-45-39a22121036e>:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  r = model.fit_generator(


Epoch 1/5
115/115 [==============================] - 588s 5s/step - loss: 4.4415 - accuracy: 0.6535 - val_loss: 2.3886 - val_accuracy: 0.7684
Epoch 2/5
115/115 [==============================] - 562s 5s/step - loss: 3.1366 - accuracy: 0.7190 - val_loss: 2.8343 - val_accuracy: 0.6894
Epoch 3/5
115/115 [==============================] - 561s 5s/step - loss: 3.1516 - accuracy: 0.7365 - val_loss: 2.3789 - val_accuracy: 0.7248
Epoch 4/5
115/115 [==============================] - 562s 5s/step - loss: 2.8436 - accuracy: 0.7589 - val_loss: 2.0137 - val_accuracy: 0.7888
Epoch 5/5
115/115 [==============================] - 559s 5s/step - loss: 3.1551 - accuracy: 0.7622 - val_loss: 2.6538 - val_accuracy: 0.7520


In [46]:
model.save('IBMDR.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [47]:
!tar -zcvf image-classification-model_new.tgz IBMDR.h5

IBMDR.h5


In [48]:
ls -1

IBMDR.h5
image-classification-model_new.tgz
testing/
training/
